In [ ]:
#load .env variables
import os
from dotenv import load_dotenv
load_dotenv()
PWD = os.getenv('PWD')
from flask import Flask, app, redirect, url_for, render_template
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
import pandas as pd

In [ ]:
app = Flask(__name__)
# attach SQL to flask
db_name = PWD+'\\database'+'\\test.db'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + db_name
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = True

# this variable, db, will be used for all SQLAlchemy commands
db = SQLAlchemy(app)


In [ ]:
class RVNUSDT(db.Model): 
    __tablename__ = 'minute_scale'
    id = db.Column(db.Integer, primary_key=True)
    # 1499040000000,      // Open time
    # "0.01634790",       // Open
    # "0.80000000",       // High
    # "0.01575800",       // Low
    # "0.01577100",       // Close
    # "148976.11427815",  // Volume
    # 1499644799999,      // Close time
    # "2434.19055334",    // Quote asset volume
    # 308,                // Number of trades
    # "1756.87402397",    // Taker buy base asset volume
    # "28.46694368",      // Taker buy quote asset volume
    # "17928899.62484339" // Ignore.
    date_created = db.Column(db.DateTime, )
    open_time = db.Column(db.String(255))
    open_ = db.Column(db.String(255))
    high_ = db.Column(db.String(255))
    low_ = db.Column(db.String(255))
    close_ = db.Column(db.String(255))
    volume_ = db.Column(db.String(255))
    close_time = db.Column(db.String(255))
    quote_asset_volume = db.Column(db.String(255))
    number_of_trades = db.Column(db.String(255))
    taker_buy_base_asset_volume = db.Column(db.String(255))
    taker_buy_quote_asset_volume = db.Column(db.String(255))
    ignore_ = db.Column(db.String(255))

    # title_id = db.Column(db.Integer)
    # name_en = db.Column(db.String(255))
    # name_ru = db.Column(db.String(255))
    # max_ep = db.Column(db.Integer)
    # from_usr = db.Column(db.String(100))
    # to_usr = db.Column(db.String(100))

    def __init__(self, 
                        date_created ,
                        open_time ,
                        open_ ,
                        high_ ,
                        low_ ,
                        close_ ,
                        volume_ ,
                        close_time ,
                        quote_asset_volume ,
                        number_of_trades ,
                        taker_buy_base_asset_volume ,
                        taker_buy_quote_asset_volume ,
                        ignore_):
        self.date_created = date_created
        self.open_time = open_time
        self.open_ = open_
        self.high_ = high_
        self.low_ = low_
        self.close_ = close_
        self.volume_ = volume_
        self.close_time = close_time
        self.quote_asset_volume = quote_asset_volume
        self.number_of_trades = number_of_trades
        self.taker_buy_base_asset_volume = taker_buy_base_asset_volume
        self.taker_buy_quote_asset_volume = taker_buy_quote_asset_volume
        self.ignore_ = ignore_

In [ ]:
db.create_all()

In [ ]:
datetime.fromtimestamp(1569412859999 / 1000)

In [ ]:
p = datetime.today() - start_dt
p.days*3600/1000 * 180
# around 500 mb of data from RVN 1m ticker

In [ ]:
from datetime import timedelta
dt_to_unix = lambda x: int(x.timestamp()*1000)
start_dt = datetime(year=2019, month=9, day=25, hour=15)
startTime = int(start_dt.timestamp()*1000)
startTime, start_dt

In [ ]:

end_dt = start_dt + timedelta(minutes=1000)
endTime = dt_to_unix(end_dt)
endTime, end_dt 

In [ ]:
from urllib.parse import urlencode
urlencode(
    {
        'a': 'b', 
        'c': 'd'
    })

In [ ]:
import requests
import json
from urllib.parse import urlencode

pair = 'RVNUSDT'
interval = '1m'
limit = 1000
startTime = 1569412800000
endTime = 1569472800000
endpoint = 'https://api.binance.com/api/v1/klines'
data = urlencode(
    {
        'symbol': pair,
        'interval': interval,
        'limit': limit,
        'startTime': startTime,
        'endTime': endTime,
    })
resource = requests.get(url=f'{endpoint}?{data}', )
data = json.loads(resource.text)
data

In [ ]:
class IterData(object): 
    '''docstring for IterData'''
    def __init__(self, init_time: datetime, pair: str='RVNUSDT', interval: str='1m', limit: int=1000):
        super(IterData, self).__init__()
        self.pair = pair
        self.init_time = init_time
        self.start_time = init_time
        self.end_time = self.start_time + timedelta(minutes=limit)
        self.interval = interval
        self.limit = limit


    def calculate_interval(self, ): 
        # calculate interval for next request from API
        # if self.start_time != self.init_time:
        self.start_time = self.start_time + timedelta(minutes=self.limit)
        self.end_time = self.start_time + timedelta(minutes=self.limit)
            

    def iter_data_get(self, ): 
        dt_to_unix = lambda x: int(x.timestamp()*1000)
        # pair = 'RVNUSDT'
        # interval = '1m'
        # limit = 1000
        # startTime = 1569412800000
        # endTime = 1569472800000
        endpoint = 'https://api.binance.com/api/v1/klines'
        data = urlencode(
            {
                'symbol': self.pair,
                'interval': self.interval,
                'limit': self.limit,
                'startTime': dt_to_unix(self.start_time),
                'endTime': dt_to_unix(self.end_time),
            })
        r = requests.get(url=f'{endpoint}?{data}', )
        self.calculate_interval()
        return json.loads(r.text)


In [ ]:
from datetime import datetime
from time import sleep
d = IterData(
    # init_time=datetime(year=2019, month=9, day=25, hour=15)
    init_time=datetime(year=2019, month=9, day=25, hour=15)
)
# print(d.end_time > datetime(year=2019, month=12, day=1, hour=0, minute=0))
try:
    # while not d.end_time > datetime(year=2019, month=10, day=3, hour=0, minute=0):
    while not d.end_time > datetime(year=2022, month=1, day=1, hour=0, minute=0):
        sleep(1)
        print(d.end_time )
        data = d.iter_data_get()
        for i in data:

            record = RVNUSDT(
                        date_created = datetime.fromtimestamp(i[0]/1000) ,
                        open_time = str(i[0]) ,
                        open_ = str(i[1]) ,
                        high_ = str(i[2]) ,
                        low_ = str(i[3]) ,
                        close_ = str(i[4]) ,
                        volume_ = str(i[5]) ,
                        close_time = str(i[6]) ,
                        quote_asset_volume = str(i[7]) ,
                        number_of_trades = str(i[8]) ,
                        taker_buy_base_asset_volume = str(i[9]) ,
                        taker_buy_quote_asset_volume = str(i[10]) ,
                        ignore_ = str(i[11])
                        )
            db.session.add(record)
finally:
    db.session.commit()

In [3]:
df = pd.read_sql(
    'SELECT * FROM minute_scale WHERE date_created > "2021-01-01 00:00:00.000000"',
    # 'SELECT * FROM minute_scale WHERE id > 990 AND id < 1010',
    'sqlite:///' + db_name, 
    index_col='id',
    )
df
# df['date_created'][990:1010]

,date_created,open_time,open_,high_,low_,close_,volume_,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore_
id,,,,,,,,,,,,,
665756,2021-01-01 00:01:00.000000,1609448460000,0.01257000,0.01257000,0.01256000,0.01256000,29084.30000000,1609448519999,365.31784800,7,29084.30000000,365.31784800,0
665757,2021-01-01 00:02:00.000000,1609448520000,0.01256000,0.01257000,0.01256000,0.01257000,36423.40000000,1609448579999,457.64174000,6,36423.40000000,457.64174000,0
665758,2021-01-01 00:03:00.000000,1609448580000,0.01255000,0.01255000,0.01255000,0.01255000,1628.50000000,1609448639999,20.43767500,1,0.00000000,0.00000000,0
665759,2021-01-01 00:04:00.000000,1609448640000,0.01256000,0.01258000,0.01256000,0.01258000,28000.00000000,1609448699999,352.03874000,3,28000.00000000,352.03874000,0
665760,2021-01-01 00:05:00.000000,1609448700000,0.01258000,0.01258000,0.01258000,0.01258000,0.00000000,1609448759999,0.00000000,0,0.00000000,0.00000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189503,2021-12-31 09:35:00.000000,1640932500000,0.09501000,0.09508000,0.09501000,0.09504000,41990.70000000,1640932559999,3991.26299500,31,41990.70000000,3991.26299500,0
1189504,2021-12-31 09:36:00.000000,1640932560000,0.09496000,0.09501000,0.09496000,0.09500000,13116.70000000,1640932619999,1246.02785500,13,10273.80000000,976.01267700,0
1189505,2021-12-31 09:37:00.000000,1640932620000,0.09497000,0.09498000,0.09493000,0.09498000,6600.10000000,1640932679999,626.83031000,10,0.00000000,0.00000000,0


In [ ]:
# delete all data from the table
db.session.query(RVNUSDT).delete() 
db.session.commit()

In [ ]:
datetime.today()

In [ ]:
pd.read_sql('SELECT * FROM table_name WHERE "column" LIKE "query"',
            'sqlite:///' + db_name, 
            # index_col='id',
            )